In [4]:
# some information about the weather for the 35 cities
# https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}

# import librairie
import requests
import json
import pandas as pd
from credentials import APIKEY 

In [5]:
# API key
APPID = APIKEY

EXCLUDE = 'current,minutely,hourly'

In [33]:
df = pd.read_csv("cities.csv")
df.head()

,Id,City,Latitude,Longitude
0,156094680,Mont Saint Michel,48.635954,-1.511460
1,297756747,St Malo,48.649518,-2.026041
2,297981358,Bayeux,49.276462,-0.702474
3,298137491,Le Havre,49.493898,0.107973
4,297518815,Rouen,49.440459,1.093966


In [7]:
params = {'exclude' : EXCLUDE ,
          'appid' : APPID ,
          'lat' : '48.6359541' ,
          'lon' : -1.511459954959514,
          'units' : 'metric'}

In [8]:
#  https://nominatim.openstreetmap.org/search?<params>
endpoint = "https://api.openweathermap.org/data/2.5/onecall?"

response =  requests.get(endpoint, params = params)
print("Response code:",response,"\n \n")

Response code: <Response [200]> 
 



In [24]:
import datetime

date = response.json()['daily'][0]['dt']
readable = datetime.datetime.fromtimestamp(date).isoformat()
readable

'2022-09-05T14:00:00'

In [9]:
params = {'exclude' : EXCLUDE ,
          'appid' : APPID ,
          'units' : 'metric'}

In [10]:
# intialisation dataframe vide
df_weather = pd.DataFrame(columns = ['Id', 'City','lat','lon','temp'])

In [11]:
# parcours la liste des 35 cités via la latitude et la longitude

for counter, (id, city, lat, lon) in enumerate(zip(df['Id'], df['City'],df['Latitude'], df['Longitude'])):

    localpoint = endpoint + f"lat={lat}&lon={lon}"
    response =  requests.get(localpoint, params = params)
    
    # Récupère la température des 7 prochains jours
    temp = []
    for i in range(1, len(response.json()['daily'])):
        temp.append((response.json()['daily'][i]['temp']['day']))
    
    # Complète le dataframe
    df_weather.loc[counter] = [id, city, lat, lon, sum(temp)/len(temp)]
    #

In [12]:
df_weather

,Id,City,lat,lon,temp
0,156094680,Mont Saint Michel,48.635954,-1.511460,23.740000
1,297756747,St Malo,48.649518,-2.026041,22.775714
2,297981358,Bayeux,49.276462,-0.702474,23.824286
3,298137491,Le Havre,49.493898,0.107973,21.451429
4,297518815,Rouen,49.440459,1.093966,23.882857
5,297417241,Paris,48.858890,2.320041,24.875714
6,297534793,Amiens,49.894171,2.295695,24.241429
7,297472400,Lille,50.636565,3.063528,23.765714
8,297508568,Strasbourg,48.584614,7.750713,24.094286
9,120791766,Chateau du Haut Koenigsbourg,48.249490,7.344296,21.480000


In [13]:
import plotly.express as px

fig = px.scatter_mapbox(df_weather,
                        lat ="lat",
                        lon = "lon",
                        color = "temp",
                        hover_name = "City",
                        zoom = 3.9,
                        mapbox_style = "carto-positron",
                        width = 800)
fig.show()

In [15]:
df_weather.to_csv("cities_weather.csv")

In [56]:
!python booking1.py

2022-09-22 16:43:36 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-09-22 16:43:36 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Windows-10-10.0.19043-SP0
2022-09-22 16:43:36 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-09-22 16:43:36 [scrapy.extensions.telnet] INFO: Telnet Password: da4152b706af3e4e
2022-09-22 16:43:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-09-22 16:43:36 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrap